# **Whole slide analysis of Cell DIVE multiplex images**

## <span style="color: red"> THIS NOTEBOOK ACTS AS A TEMPLATE AND WILL NOT SAVE CHANGES UNLESS YOU SAVE A COPY TO YOUR OWN FOLDER. USE `FILE->DOWNLOAD` IN THE TASKBAR AT THE VERY TOP OF THIS PAGE TO SAVE A COPY OF THE NOTEBOOK. THEN, USING THE SIDEBAR TO THE RIGHT OPEN THE NEW COPY OF THE NOTEBOOK BEFORE YOU GET STARTED (RUNNING ON WSL C: AND D: DRIVES SHOULD BE LOCATED UNDER `drives/c` AND `drives/d`, RESPECTIVELY) </span> 

Run through this notebook **step-by-step** and adjust the code if necessary at any point in the process. The explanation and comments througout the notebook will help guide you through the process. There will also be pointers as to modifications you might want to consider.

## Dependencies
We load all the necessary libraries used for the processing, analysis, clustering and visualisation in this notebook.

In [ ]:
library(geometry)
library(uwot)
library(celldiveUtils)
library(stringr)
library(patchwork)

## 1. Define data location and setup directory structure

### 1.1 Set data directory

The first step is to set the *data/base directory* `base_dir` that contains a folder called `segmentation` with the segmentation and quantification results. The content of the *base* directory would look something like the example below. We will also save the results and all the pipeline outputs in this folder as well.

```
├── ome_tiff
├── image_data
└── segmentation
    ├── cell_table
    ├── cell_table_no_qc
    ├── deepcell_input
    ├── deepcell_output
    ├── deepcell_output_no_qc
    ├── deepcell_visualization
    └── deepcell_visualization_no_qc
    
```

In order to define the *data/base directory* we have two options:
* set `set_base_dir_method="Manual"` and manually define the data/base directory below via `data_dir=`
* set `set_base_dir_method="Relative"` which assume that this notebook is saved in a `notebook` folder alongside the `ome_tiff` in the same parent directory

The last option `Relative` ensures greatest reproducibility and transferability as the notebook is saved alongside the data and all path are defined relative to it.

**NOTE: In `WSL` the `C:` drive, `D:` drive, etc are mounted and located at `/mnt/c`, `/mnt/d`, etc, respectively.**

In [ ]:
set_base_dir_method <- "Relative"

In [ ]:
if (set_base_dir_method == "Relative") {
    notebook_dir <- getwd()
    data_dir <- fs::path_dir(notebook_dir)
} else if (set_base_dir_method == "Manual") {
    data_dir = "/path/to/data/directory"
}

In [ ]:
# set the base directory
base_dir = data_dir

### 1.2 Setup intermediate and results paths

Next, we create results directories to save the plots, etc. such that we have the following folder structure in the `base directory`:

```
├── ome_tiff
├── image_data
├── segmentation
└── analysis
    
```

Here, we set all the directory names to be created.

In [ ]:
results_dir = fs::path_join(c(base_dir, "analysis"))
cell_table_dir = fs::path_join(c(base_dir, "segmentation","cell_table_no_qc"))
cell_table_dir_qc = fs::path_join(c(base_dir, "segmentation", "cell_table"))

Then, we check if the directories exist and if not create them.

In [ ]:
for (directory in c(results_dir)) {
    dir.create(directory,showWarnings = FALSE)
}

## SECTION 1: GENERATE UMAPs, INTENSITIES, NUMBER OF CELLS POST-QC - SAVE AS GUT CELLS.RDS - optimise for samples done as per Ark

In [ ]:
input_data = "normalized" #or "transformed"

In [ ]:
if (input_data == "normalized") {
    file_pattern = ".*_normalized_"
} else if (input_data == "transformed") {
    file_pattern = ".*_arcsinh_transformed_"
}

In [ ]:
cell_table_files = list.files(cell_table_dir_qc, pattern = file_pattern)

In [ ]:
tissue_names  = stringr::str_match(cell_table_files,str_glue("cell_table_{file_pattern}(.*?)_region_001.csv"))[,2]

In [ ]:
tissue_names[2] = "GI_6717"

In [ ]:
cell_tables = setNames(cell_table_files, tissue_names)

In [ ]:
fs::path_join(c("test_", cell_tables[tissue_names[1]]))

In [ ]:
cell_tables

In [ ]:
# cells <- list()

# ##these samples have CD14, and CD34, but not CDH11, or PANCADHERIN
# cells$metadata <- purrr::map(c('GI_6645', 'GI_6846'), function(libname) {
#     data.table::fread(fs::path_join(c(cell_table_dir_qc,str_glue('cell_table_size_normalized_{libname}_region_001.csv')))) %>% 
#         dplyr::rename(x = 'centroid-1', y='centroid-0', CellNum=label) %>%
#         dplyr::mutate(CellID = paste0('Cell', CellNum, '_', libname)) %>%
#         dplyr::mutate(LibraryID = libname)  %>%
#         dplyr::select(-c(CD14, CD34)) %>%
#         dplyr::select(CellID, LibraryID, x, y, cell_size, DAPI_INIT,DAPI_FINAL, everything()) %>% 
#         data.frame()
# }) %>% 
#     dplyr::bind_rows()


# GI_6717 <- data.table::fread(fs::path_join(c(cell_table_dir_qc,'cell_table_size_normalized_GI_6717_3_region_001.csv'))) %>% dplyr::rename(x = 'centroid-1', y='centroid-0', CellNum=label) %>% dplyr::mutate(CellID = paste0('Cell', CellNum, '_', "GI_6717")) %>%
#     dplyr::mutate(LibraryID = "GI_6717")  %>%
#     dplyr::select(-c(CDH11, PANCADHERIN)) %>%
#     dplyr::select(CellID, LibraryID, x, y, cell_size, DAPI_INIT,DAPI_FINAL, everything()) %>% 
#     data.frame()

# cells$metadata <- dplyr::bind_rows(cells$metadata, GI_6717) 

In [ ]:
cells <- list()

cells$metadata <-purrr::map(tissue_names, function(tissue_name) {
    data.table::fread(fs::path_join(c(cell_table_dir_qc,cell_tables[tissue_name]))) %>% 
        dplyr::rename(x = 'centroid-1', y='centroid-0', CellNum=label) %>%
        dplyr::mutate(CellID = paste0('Cell', CellNum, '_', tissue_name)) %>%
        dplyr::mutate(LibraryID = tissue_name)  %>%
        dplyr::select(CellID, LibraryID, x, y, cell_size, DAPI_INIT,DAPI_FINAL, everything()) %>% 
        data.frame()
}) %>% 
    dplyr::bind_rows() %>%
    dplyr::select_if(~ !any(is.na(.))) %>% 
    dplyr::mutate(x_img = x, y_img = y) %>% 
    dplyr::mutate(y = dplyr::case_when(
        LibraryID == 'GI_6645' ~ y_img,
        LibraryID == 'GI_6846' ~ y_img + 8500,
        LibraryID == 'GI_6717' ~ y_img + 16000
    ))

In [ ]:
cells$metadata %>% 
    split(.$LibraryID) %>% 
    purrr::imap(function(.SD, .name) {
        .SD %>% dplyr::arrange(-y) %>% head(3)
    })


# ## Offset y so we can plot images in same space 
# cells$metadata <- cells$metadata %>% 
#     dplyr::mutate(x_img = x, y_img = y) %>% 
#     dplyr::mutate(y = dplyr::case_when(
#         LibraryID == 'GI_6645' ~ y_img,
#         LibraryID == 'GI_6846' ~ y_img + 8500,
#         LibraryID == 'GI_6717' ~ y_img + 16000
#     ))

head(cells$metadata)

In [ ]:
# nrow(cells_alt$metadata)

In [ ]:
nrow(cells$metadata)

In [ ]:
# tail(sort_by(cells_alt$metadata, list(cells_alt$metadata$LibraryID, cells_alt$metadata$CellNum)))

In [ ]:
# tail(sort_by(cells$metadata, list(cells$metadata$LibraryID, cells$metadata$CellNum)))

In [ ]:
# sum(!(sort_by(cells_alt$metadata, list(cells_alt$metadata$LibraryID, cells_alt$metadata$CellNum)) == sort_by(cells$metadata, list(cells$metadata$LibraryID, cells$metadata$CellNum))))

In [ ]:
table(cells$metadata$LibraryID)

In [ ]:
##plot samples on a umap
celldiveUtils::do_scatter(data.frame(x = cells$metadata$y, y = -cells$metadata$x), cells$metadata, 'LibraryID')

In [ ]:
# celldiveUtils::do_scatter(data.frame(x = cells_alt$metadata$y, y = -cells_alt$metadata$x), cells_alt$metadata, 'LibraryID')

In [ ]:
##plot number of cells per sample
data.table::data.table(cells$metadata)[, .N, by = LibraryID] %>% 
    ggplot2::ggplot(ggplot2::aes(LibraryID, N)) + 
        ggplot2::geom_bar(stat = 'identity') + 
        ggplot2::coord_flip() + 
        ggplot2::theme_bw(base_size = 14)

In [ ]:
# ##plot number of cells per sample
# data.table::data.table(cells_alt$metadata)[, .N, by = LibraryID] %>% 
#     ggplot2::ggplot(ggplot2::aes(LibraryID, N)) + 
#         ggplot2::geom_bar(stat = 'identity') + 
#         ggplot2::coord_flip() + 
#         ggplot2::theme_bw(base_size = 14)

## SECTION 2: SHUFFLE INTENSITY INTO ITS OWN SLOT

In [ ]:
metadata_columns <- c("LibraryID","x","y","cell_size","CellNum", "area", "eccentricity", "major_axis_length", "minor_axis_length", "perimeter", "convex_area", "equivalent_diameter", "major_minor_axis_ratio", "perim_square_over_area", "major_axis_equiv_diam_ratio", "convex_hull_resid", "centroid_dif", "num_concavities", "fov", "mask_type", "x_img", "y_img")
channels_keep <- setdiff(colnames(cells$metadata),metadata_columns)
cells$intensity <- cells$metadata[, channels_keep]
cells$intensity <- cells$intensity %>% tibble::column_to_rownames(var = "CellID")

In [ ]:
metadata_columns <- c("CellID","LibraryID","x","y","cell_size","CellNum", "area", "eccentricity", "major_axis_length", "minor_axis_length", "perimeter", "convex_area", "equivalent_diameter", "major_minor_axis_ratio", "perim_square_over_area", "major_axis_equiv_diam_ratio", "convex_hull_resid", "centroid_dif", "num_concavities", "fov", "mask_type", "x_img", "y_img")
cells$metadata <- cells$metadata[, metadata_columns]

In [ ]:
###NOTE: WE DO NOT NEED TO DO FURTHER QC on cells AS THE ARK PIPELINE HAS ALREADY PERFORMED QC ON AREA, AND DAPI-DEPENDENT STEP HAS ALREADY BEEN IMPLEMENTED IN THE SEGMENTATION STEP.
#we do not need to run 'do_norm(cells)' as we have already size normalised within the Ark pipeline

#QC channels
channels_keep <- setdiff(colnames(cells$intensity), c('PSTAT1', 'CD11C', 'CD34', 'PANCADHERIN', 'CDH11', 'DAPI_INIT', 'DAPI_FINAL', 'MERTK')) ## these are very low quality 
cells$intensity <- cells$intensity[, channels_keep]

#viz channels (density) - the do_scale function has been altered within utils_celldive.r to account for the fact our cells$intensity has already been normalised
cells <- celldiveUtils::do_scale(cells, z_thresh=3, within_batch=TRUE)

In [ ]:
cells$z %>% 
    data.frame() %>% 
    tibble::rownames_to_column('CellID') %>% 
    cbind(LibraryID = cells$metadata$LibraryID) %>% 
    tidyr::gather(key, val, all_of(colnames(cells$intensity))) %>% 
    ggplot2::ggplot(ggplot2::aes(val, key, color = LibraryID)) + 
        ggridges::geom_density_ridges2(fill = NA)

In [ ]:
cells$intensity %>% 
    tibble::rownames_to_column('CellID') %>% 
    cbind(LibraryID = cells$metadata$LibraryID) %>% 
    tidyr::gather(key, val, all_of(colnames(cells$intensity))) %>% 
    ggplot2::ggplot(ggplot2::aes(log1p(val), key, color = LibraryID)) + 
        ggridges::geom_density_ridges2(fill = NA)

In [ ]:
#viz intensities (space)
features_plot <- colnames(cells$intensity)
suppressWarnings({
    celldiveUtils::plotFeatures(
        t(cells$z), 
        data.frame(x = cells$metadata$y, y = -cells$metadata$x), 
        features_plot, 
        nrow = floor(sqrt(length(features_plot))),
        no_guide = TRUE
    )    
})

saveRDS(cells, fs::path_join(c(results_dir, 'gut_cells.rds')))

## SECTION 3: MAKE SPOTS

In [ ]:
###AHHH we are missing 'area', we need to ask Jonas to re-run the segmentation, with some parameter changes in Section 5.1 changes
###Specifically, we need to set nuclear_counts to True, and fast_extraction to False
###ISSUES WITH GAGARIN - this is super annoying - we need to install "spatula" -> issues with this locally so move to HPC

###Niche analysis - do this on the cluster with the following R package
#module load R/4.1.0-foss-2021a
cells <- readRDS(fs::path_join(c(results_dir, 'gut_cells.rds')))
spots <- celldiveUtils::make_spots(cells)
saveRDS(spots, fs::path_join(c(results_dir, 'gut_spots.rds')))

## SECTION 4: CLUSTERING SPOTS

In [ ]:
cells <- readRDS(fs::path_join(c(results_dir, 'gut_cells.rds')))
spots <- readRDS(fs::path_join(c(results_dir, 'gut_spots.rds')))

system.time({
    spots <- spots %>% 
        celldiveUtils::do_norm() %>% 
        celldiveUtils::do_scale(3, TRUE) %>% 
        celldiveUtils::do_pca() %>% 
        celldiveUtils::do_umap('V', 'U0') %>% 
        celldiveUtils::harmonize() %>% 
        celldiveUtils::do_umap('H', 'U1') %>% 
        identity()
})

# fig.size(5, 15)
(
    celldiveUtils::do_scatter(spots$U1$embedding, spots$metadata, 'LibraryID') | 
    celldiveUtils::do_scatter(spots$U1$embedding, spots$metadata, 'LibraryID', dplyr::quo(LibraryID)) 
) + 
patchwork::plot_layout(widths = c(1, 3))

pdf(fs::path_join(c(results_dir, "features_spots_clustering_umap.pdf")), height=16, width=24)
features_plot <- colnames(spots$z)
suppressWarnings({
    celldiveUtils::plotFeatures(
        t(spots$z), 
        spots$U1$embedding, 
        features_plot, 
        nrow = floor(sqrt(length(features_plot))),
        no_guide = TRUE)    
})
dev.off()

system.time({
    spots <- spots %>% 
        celldiveUtils::do_louvain('U1', c(.1, .4, .8, 1.2)) %>% 
        celldiveUtils::do_markers()
})

## Only plot up to 33 colors 
clusters_plot <- names(which(spots$Clusters %>% map(table) %>% map_int(length) <= length(celldiveUtils:::colors_overload)))

pdf(fs::path_join(c(results_dir, "spots_clustering_umap.pdf")), height=30, width=30)
# fig.size(4 * length(clusters_plot), 15)
map(clusters_plot, function(.name) {
    celldiveUtils::do_scatter(spots$U1$embedding, spots$Clusters, .name) |
    celldiveUtils::do_scatter(data.frame(x = spots$metadata$y, y = -spots$metadata$x), spots$Clusters, .name, do_labels = FALSE)
}) %>% 
    reduce(`/`)
dev.off()

saveRDS(spots, fs::path_join(c(results_dir, 'gut_spots_post_clustering.rds')))

## SECTION 5: CLUSTERING CELLS

In [ ]:
cells <- readRDS(fs::path_join(c(results_dir, 'gut_cells.rds')))

system.time({
    cells <- cells %>% 
        celldiveUtils::do_norm() %>% 
        celldiveUtils::do_scale(3, TRUE) %>% 
        celldiveUtils::do_pca() %>% 
        celldiveUtils::do_umap('V', 'U0') %>% 
        celldiveUtils::harmonize() %>% 
        celldiveUtils::do_umap('H', 'U1') %>% 
        identity()
})

# fig.size(5, 15)
with(cells, {
    (
        celldiveUtils::do_scatter(U1$embedding, metadata, 'LibraryID') | 
        celldiveUtils::do_scatter(U1$embedding, metadata, 'LibraryID', dplyr::quo(LibraryID)) 
    ) + 
    plot_layout(widths = c(1, 3))
})

pdf(fs::path_join(c(results_dir, "features_cells_clustering_umap.pdf")), height=16, width=24)
# fig.size(16, 24)
features_plot <- colnames(cells$z)
suppressWarnings({
    celldiveUtils::plotFeatures(
        t(cells$z), 
        cells$U1$embedding, 
        features_plot, 
        nrow = floor(sqrt(length(features_plot))),
        no_guide = TRUE
    )    
})
dev.off()

system.time({
    cells <- cells %>% 
        celldiveUtils::do_louvain('U1', c(.1, .4, .8, 1.2)) %>% 
        celldiveUtils::do_markers()
})

pdf(fs::path_join(c(results_dir, "cells_clustering_umap.pdf")), height=30, width=30)
## Only plot up to 33 colors 
with(cells, {
    clusters_plot <- names(which(Clusters %>% map(table) %>% map_int(length) <= length(celldiveUtils:::colors_overload)))
    # fig.size(4 * length(clusters_plot), 15)
    map(clusters_plot, function(.name) {
        celldiveUtils::do_scatter(U1$embedding, Clusters, .name) |
        celldiveUtils::do_scatter(data.frame(x = metadata$y, y = -metadata$x), Clusters, .name, do_labels = FALSE)
    }) %>% 
        reduce(`/`)
    
})
dev.off()

saveRDS(cells, fs::path_join(c(results_dir, 'gut_cells_post_clustering.rds')))

## SECTION 6: LABEL NICHES

## SECTION 7: PLOT MARKERS (UMAPs/HEATMAPS)

In [ ]:
spots <- readRDS(fs::path_join(c(results_dir, 'gut_spots_post_clustering.rds')))
cells <- readRDS(fs::path_join(c(results_dir, 'gut_cells_post_clustering.rds')))

pdf(fs::path_join(c(results_dir, "post_plot_clusters.pdf")), height=30, width=30)
with(spots, {
    clusters_plot <- names(which(Clusters %>% map(table) %>% map_int(length) <= length(celldiveUtils:::colors_overload)))
    # fig.size(4 * length(clusters_plot), 15)
    map(clusters_plot, function(.name) {
        celldiveUtils::do_scatter(U1$embedding, Clusters, .name) |
        celldiveUtils::do_scatter(data.frame(x = metadata$y, y = -metadata$x), Clusters, .name, do_labels = FALSE)
    }) %>% 
        reduce(`/`)
    
})
dev.off()

pdf(fs::path_join(c(results_dir, "clusters.pdf")), height = 10, width = 10)
with(spots, celldiveUtils::do_scatter(U1$embedding, Clusters, 'Clust0.4'))
dev.off()

pdf(fs::path_join(c(results_dir, "clusters_individual.pdf")), height = 15, width = 10)
with(spots, {
    celldiveUtils::do_scatter(U1$embedding, Clusters, 'Clust0.4', dplyr::quo(`Clust0.4`), do_labels=FALSE, nrow=6)       
})
dev.off()

pdf(fs::path_join(c(results_dir, "markers_across_clusters.pdf")), height = 4, width = 15)
celldiveUtils::plotFeatures(t(spots$z), spots$U1$embedding, c('ASMA', 'CD31', 'CD90', 'CD146'), nrow = 1,no_guide = TRUE) %>% plot()
celldiveUtils::plotFeatures(t(spots$z), spots$U1$embedding, c('CCL19', 'CD3', 'CD4', 'CD45'), nrow = 1,no_guide = TRUE) %>% plot() 
dev.off()

pdf(fs::path_join(c(results_dir, "stroma_vs_epi_across_clusters.pdf")), height = 4, width = 8)
celldiveUtils::plotFeatures(t(spots$z), spots$U1$embedding, c('VIM', 'PANCK'), nrow = 1, no_guide = TRUE) %>% plot() 
dev.off()

pdf(fs::path_join(c(results_dir, "heatmap_multiple_res.pdf")), height = 4, width = 8)
celldiveUtils::plot_heatmap(spots$Markers$`Clust0.1`, c('ASMA', 'CD31', 'CD90', 'CD146', 'CCL19', 'CD3', 'CD4', 'CD45', 'VIM', 'PANCK'), TRUE)
celldiveUtils::plot_heatmap(spots$Markers$`Clust0.4`, c('ASMA', 'CD31', 'CD90', 'CD146', 'CCL19', 'CD3', 'CD4', 'CD45', 'VIM', 'PANCK'), TRUE)
celldiveUtils::plot_heatmap(spots$Markers$`Clust0.8`, c('ASMA', 'CD31', 'CD90', 'CD146', 'CCL19', 'CD3', 'CD4', 'CD45', 'VIM', 'PANCK'), TRUE)
celldiveUtils::plot_heatmap(spots$Markers$`Clust1.2`, c('ASMA', 'CD31', 'CD90', 'CD146', 'CCL19', 'CD3', 'CD4', 'CD45', 'VIM', 'PANCK'), TRUE)
dev.off()

## SECTION 8: NICHE HEATMAP

In [ ]:
if ('Niche' %in% colnames(spots$metadata)) 
    spots$metadata$Niche <- NULL

spots$metadata$Cluster <- paste0('C', spots$Clusters$`Clust1.2`)

spots$metadata <- spots$metadata %>% 
    dplyr::mutate(Niche = dplyr::case_when(
        Cluster %in% paste0('C', c(12,1,0,26,8,18)) ~ 'Lympho_Vascular',
        Cluster %in% paste0('C', c(2,5,14,17,16)) ~ 'Perivascular',
        Cluster %in% paste0('C', c(13,32,4,3,27,11,10,30,9,28)) ~ 'Other',
        Cluster %in% paste0('C', c(29,19,23,21,25,31)) ~ 'Epithelial',
        Cluster %in% paste0('C', c(6,22,7,20,15,24)) ~ 'Lymphoid'
    )) %>% 
    dplyr::mutate(Niche_Broad = gsub('\\d+', '', Niche))

pdf(fs::path_join(c(results_dir, "niches_umap.pdf")), width=8, height=8)
# fig.size(8, 12)
with(spots, {
    celldiveUtils::do_scatter(U1$embedding, metadata, 'Niche', do_labels = TRUE)    
})
dev.off()

pdf(fs::path_join(c(results_dir, "niches.pdf")), width=12, height=8)
# fig.size(8, 12)
with(spots, {
    celldiveUtils::do_scatter(
        data.frame(x = metadata$y, y = -metadata$x),
        metadata, 
        'Niche', dplyr::quo(Niche_Broad), 
        do_labels = FALSE, 
        nrow = 2
    )    
})
dev.off()

pdf(fs::path_join(c(results_dir, "niche_heatmaps.pdf")), width=8, height=10)
# fig.size(8, 10)
with(spots, presto::wilcoxauc(t(z), metadata$Niche)) %>% 
    celldiveUtils::plot_heatmap(.scale=TRUE)
dev.off()

saveRDS(spots, fs::path_join(c(results_dir, 'gut_spots_post_niche_annotation.rds')))

## SECTION 9: FIBROBLAST ANALYSIS HEATMAP

## SECTION 10: LOCALISATION ENRICHMENT